This notebook is made to get a better understanding of what data we actually have to do this project

In [ ]:
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

Loading yaml file where the locations of the data is being stored

In [ ]:
import os 
os.getcwd()

In [ ]:
def get_config():
    with open("../data/config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

Loading the 3 files (center label, diagnosis label, and feature vectors)

In [ ]:
config = get_config()
feature_vectors = pd.read_csv(config['feature_vectors'], header=None)
feature_vectors = feature_vectors.set_axis(list(range(1, 36)), axis=1)
center_label = pd.read_csv(config['center_label'], header=None)
center_label.rename(columns={0: "center_label"}, inplace=True)
diagnosis_label = pd.read_csv(config['diagnosis_label'], header=None)
diagnosis_label.rename(columns={0: "diagnosis_label"}, inplace=True)


In [ ]:
df = pd.concat([center_label, diagnosis_label, feature_vectors], axis=1)

In [ ]:
count_by_center = df.iloc[:, 0:2].groupby(["center_label"], as_index=False).count()
count_by_diagnosis = df.iloc[:, 0:2].groupby(["diagnosis_label"], as_index=False).count()
count_by_center_and_diagnosis = df.iloc[:, 0:3].groupby(['center_label', 'diagnosis_label'], as_index=False).count()

In [ ]:
def create_piechart(labels, data):
    fig, ax = plt.subplots()
    ax.pie(data, labels=labels, autopct='%1.1f%%')


In [ ]:
label = count_by_center['center_label']
data = count_by_center['diagnosis_label']
create_piechart(label, data)

In [ ]:
label = count_by_diagnosis['diagnosis_label']
data = count_by_diagnosis['center_label']
create_piechart(label, data)

In [ ]:
count_by_center_and_diagnosis["label"] = count_by_center_and_diagnosis['center_label'].astype(str) +"-"+ count_by_center_and_diagnosis["diagnosis_label"]
label = count_by_center_and_diagnosis["label"]
data = count_by_center_and_diagnosis.iloc[:,2]
create_piechart(label, data)

In [ ]:
first_2_eigenvectors = df.loc[:, :2]
PD_df = first_2_eigenvectors[first_2_eigenvectors['diagnosis_label'] == 'PD']

groups = PD_df.groupby('center_label')
for name, group in groups:
    print(name)
    plt.scatter(group[1], group[2], label=name)
plt.title("PD diagnosis patients from different centres")
plt.xlabel("first eigenvector")
plt.ylabel("second eigenvector")
plt.legend()


# checking to see if the data follows some kind of distribution
Looking at the graphs below it looks like the feature vectors do follow a normal distribution

In [ ]:
def DS_Q_Q_Plot(y, est = 'robust', **kwargs):
    """
    *
    Function DS_Q_Q_Plot(y, est = 'robust', **kwargs)
    
       This function makes a normal quantile-quantile plot (Q-Q-plot), also known
       as a probability plot, to visually check whether data follow a normal distribution.
    
    Requires:            - 
    
    Arguments:
      y                  data array
      est                Estimation method for normal parameters mu and sigma:
                         either 'robust' (default), or 'ML' (Maximum Likelihood),
                         or 'preset' (given values)
      N.B. If est='preset' than the *optional* parameters mu, sigma must be provided:
      mu                 preset value of mu
      sigma              preset value of sigma
      
    Returns:
      Estimated mu, sigma, n, and expected number of datapoints outside CI in Q-Q-plot.
      Q-Q-plot
      
    Author:            M.E.F. Apol
    Date:              2020-01-06, revision 2022-08-30
    """
    
    import numpy as np
    from scipy.stats import iqr # iqr is the Interquartile Range function
    import matplotlib.pyplot as plt
    
    # First, get the optional arguments mu and sigma:
    mu_0 = kwargs.get('mu', None)
    sigma_0 = kwargs.get('sigma', None)
    
    n = len(y)
    
    # Calculate order statistic:
    y_os = np.sort(y)
  
    # Estimates of mu and sigma:
    # ML estimates:
    mu_ML = np.mean(y)
    sigma2_ML = np.var(y)
    sigma_ML = np.std(y) # biased estimate
    s2 = np.var(y, ddof=1)
    s = np.std(y, ddof=1) # unbiased estimate
    # Robust estimates:
    mu_R = np.median(y)
    sigma_R = iqr(y)/1.349

    # Assign values of mu and sigma for z-transform:
    if est == 'ML':
        mu, sigma = mu_ML, s
    elif est == 'robust':
        mu, sigma = mu_R, sigma_R
    elif est == 'preset':
        mu, sigma = mu_0, sigma_0
    else:
        print('Wrong estimation method chosen!')
        return()
        
    print('Estimation method: ' + est)
    print('n = {:d}, mu = {:.4g}, sigma = {:.4g}'.format(n, mu,sigma))
    
    # Expected number of deviations (95% confidence level):
    n_dev = np.round(0.05*n)
    
    print('Expected number of data outside CI: {:.0f}'.format(n_dev))
         
    # Perform z-transform: sample quantiles z.i
    z_i = (y_os - mu)/sigma

    # Calculate cumulative probabilities p.i:
    i = np.array(range(n)) + 1
    p_i = (i - 0.5)/n

    # Calculate theoretical quantiles z.(i):
    from scipy.stats import norm
    z_th = norm.ppf(p_i, 0, 1)

    # Calculate SE or theoretical quantiles:
    SE_z_th = (1/norm.pdf(z_th, 0, 1)) * np.sqrt((p_i * (1 - p_i)) / n)

    # Calculate 95% CI of diagonal line:
    CI_upper = z_th + 1.96 * SE_z_th
    CI_lower = z_th - 1.96 * SE_z_th

    # Make Q-Q plot:
    plt.plot(z_th, z_i, 'o', color='k', label='experimental data')
    plt.plot(z_th, z_th, '--', color='r', label='normal line')
    plt.plot(z_th, CI_upper, '--', color='b', label='95% CI')
    plt.plot(z_th, CI_lower, '--', color='b')
    plt.xlabel('Theoretical quantiles, $z_{(i)}$')
    plt.ylabel('Sample quantiles, $z_i$')
    plt.title('Q-Q plot (' + est + ')')
    plt.legend(loc='best')
    plt.show()
    pass;

In [ ]:
def DS_Q_Q_Hist(y, est='robust', **kwargs):
    """
    *
    Function DS_Q_Q_Hist(y, est='robust', **kwargs)
    
       This function makes a histogram of the data and superimposes a fitted normal
       distribution.
       
    Requires:            - 
    
    Arguments:
      y                  data array
      est                Estimation method for normal parameters mu and sigma:
                         either 'robust' (default), or 'ML' (Maximum Likelihood),
                         or 'preset' (given values)
      N.B. If est='preset' than the optional parameters mu, sigma must be provided:
      mu                 preset value of mu
      sigma              preset value of sigma
    
    Returns:
      Estimations of mu and sigma
      Histogram of data with estimated normal distribution superimposed
      
    Author:            M.E.F. Apol
    Date:              2020-01-06
    """
    
    import numpy as np
    from scipy.stats import iqr # iqr is the Interquartile Range function
    from scipy.stats import norm
    import matplotlib.pyplot as plt
    
    # First, get the optional arguments mu and sigma:
    mu_0 = kwargs.get('mu', None)
    sigma_0 = kwargs.get('sigma', None)
    
    n = len(y)
    
    # Estimates of mu and sigma:
    # ML estimates:
    mu_ML = np.mean(y)
    sigma2_ML = np.var(y) # biased estimate
    sigma_ML = np.std(y) 
    s2 = np.var(y, ddof=1) # unbiased estimate
    s = np.std(y, ddof=1) 
    # Robust estimates:
    mu_R = np.median(y)
    sigma_R = iqr(y)/1.349

    # Assign values of mu and sigma for z-transform:
    if est == 'ML':
        mu, sigma = mu_ML, s       
    elif est == 'robust':
        mu, sigma = mu_R, sigma_R
    elif est == 'preset':
        mu, sigma = mu_0, sigma_0
    else:
        print('Wrong estimation method chosen!')
        return()
    print('Estimation method: ' + est)
    print('mu = {:.4g}, sigma = {:.4g}'.format(mu,sigma))
        
    # Calculate the CLT normal distribution:
    x = np.linspace(np.min(y), np.max(y), 501)
    rv = np.array([norm.pdf(xi, loc = mu, scale = sigma) for xi in x])
    
    # Make a histogram with corresponding normal distribution:
    plt.hist(x=y, density=True, bins='auto', 
             color='darkgrey',alpha=1, rwidth=1, label='experimental')
    plt.plot(x, rv, 'r', label='normal approximation')
    plt.grid(axis='y', alpha=0.5)
    plt.xlabel('Values, $y$')
    plt.ylabel('Probability $f(y)$')
    plt.title('Histogram with corresponding normal distribution (' + est + ')')
    plt.legend(loc='best')
    plt.show()
    pass;

In [ ]:
columns = df.iloc[:, 2:].columns
for i in columns:
    DS_Q_Q_Plot(df[i])
    DS_Q_Q_Hist(df[i])

In [ ]:

normalized_df = (df.iloc[:, 2:]-df.iloc[:, 2:].mean())/df.iloc[:, 2:].std()
normalized_df

In [ ]:
len(df)
for i in range(len(df)):

    DS_Q_Q_Plot(normalized_df.iloc[i], est="ML")
    DS_Q_Q_Hist(normalized_df.iloc[i], est="ML")